### Hurricane Irma 
September 8, 2017, 12 Zulu, Created @ ~9AM EST data forecasts

Source: https://www.nhc.noaa.gov/gis/archive_forecast_results.php?id=al11&year=2017&name=Hurricane%20IRMA  
File1: al112017_5day_037.zip (8:59 AM)    
File2: al112017_fcst_037.zip (9:00 AM)

In [23]:
import pandas as pd 
import numpy as np
import datetime

import geojson
from geojson import Point, Feature, FeatureCollection, dump
from shapely.geometry import shape, Point
from geopy.distance import geodesic
import tempfile
import fiona

import warnings
warnings.filterwarnings('ignore')

Import data with fiona

In [257]:
path = fiona.open("./al112017_5day_037/al112017-037_5day_lin.shp")

print(path.schema)
# {'geometry': 'LineString', 'properties': OrderedDict([(u'FID', 'float:11')])}
#first feature of the shapefile
first_path = path.next()
print(first_path) # (GeoJSON format)
# {'geometry': {'type': 'LineString', 'coordinates': [(0.0, 0.0), (25.0, 10.0), (50.0, 50.0)]}, 'type': 'Feature', 'id': '0', 'properties': OrderedDict([(u'FID', 0.0)])}


{'properties': OrderedDict(), 'geometry': 'LineString'}
{'type': 'Feature', 'id': '0', 'geometry': {'type': 'LineString', 'coordinates': [(-73.8, 21.7), (-75.7, 22.1), (-77.8, 22.6), (-79.4, 23.3), (-80.4, 24.5), (-81.5, 28.0), (-84.0, 33.0), (-87.0, 36.0)]}, 'properties': OrderedDict()}


Demo export JSON

In [259]:
path_features = []
path_features.append(Feature(geometry = first_path['geometry'], properties={"hurricane": "Irma"}))

# add more features...
for i in range(0, 6):
    point = Point(first_path['geometry']['coordinates'][i])
    day = str(datetime.date(2008, 9, 10) + datetime.timedelta(i))
    path_features.append(Feature(geometry = point, properties = {"day": day}))

with open('./geojson/path.geojson', 'w') as f:
    dump(FeatureCollection(path_features), f)

Cone

In [85]:
cone = fiona.open("./al112017_5day_037/al112017-037_5day_pgn.shp")

print(cone.schema)
first_cone = cone.next()
# {'geometry': {'type': 'LineString', 'coordinates': [(0.0, 0.0), (25.0, 10.0), (50.0, 50.0)]}, 'type': 'Feature', 'id': '0', 'properties': OrderedDict([(u'FID', 0.0)])}

{'properties': OrderedDict(), 'geometry': 'Polygon'}


In [86]:
cone_features = []
cone_features.append(Feature(geometry = first_cone['geometry'], properties={"shape": "Prediction"})) 

with open('./geojson/cone.geojson', 'w') as f:
    dump(FeatureCollection(cone_features), f)

Radius

In [241]:
radius = fiona.open("./al112017_fcst_037/al112017_2017090809_forecastradii.shp")
first_radius = radius[0]
print(first_radius)

{'type': 'Feature', 'id': '0', 'geometry': {'type': 'Polygon', 'coordinates': [[(-73.774521, 24.361803), (-73.723541, 24.360977), (-73.6726, 24.359324), (-73.621689, 24.356844), (-73.570839, 24.353542), (-73.520065, 24.349411), (-73.469376, 24.344463), (-73.4188, 24.33869), (-73.368347, 24.3321), (-73.318039, 24.324692), (-73.267876, 24.316469), (-73.217888, 24.307434), (-73.168083, 24.297592), (-73.118492, 24.28694), (-73.069115, 24.275488), (-73.019981, 24.263237), (-72.971092, 24.250193), (-72.92247, 24.236357), (-72.87413, 24.221735), (-72.826088, 24.206329), (-72.778366, 24.190149), (-72.730965, 24.173199), (-72.683922, 24.155481), (-72.637222, 24.137005), (-72.590904, 24.117775), (-72.544975, 24.097795), (-72.499443, 24.077074), (-72.454338, 24.05562), (-72.40966, 24.033438), (-72.365425, 24.010534), (-72.321655, 23.986917), (-72.278358, 23.962595), (-72.235542, 23.937574), (-72.19323, 23.911865), (-72.151428, 23.885471), (-72.110161, 23.858406), (-72.069435, 23.830675), (-72.029

In [242]:
radius_features = []
for i in range(len(radius)):
    rad_num = radius[i]
    radius_features.append(Feature(geometry = rad_num['geometry'], properties={"shape": "Prediction"})) 

with open('./geojson/radii.geojson', 'w') as f:
    dump(FeatureCollection(radius_features), f)

In [267]:
# 3, 3, 3, 3, 2, 2 of radii
def apply_radius_list(radius_features):
    radius_df = {}
    for i in [0, 1, 2, 3]:
        radius_df[i] = [radius_features[j]['geometry']['coordinates'][0] for j in range(i, i + 3)]
    for i in [4, 5]:
        radius_df[i] = [radius_features[j]['geometry']['coordinates'][0] for j in range(i, i + 2)]
    return radius_df

Containment

In [164]:
point = Point((-79.34144, 22.12434))
polygon = shape(first_radius['geometry'])
if(polygon.contains(point)):
    print('Found containing polygon')

Found containing polygon


Distance. Apart 69 miles assumption, using geodesic

In [165]:
point = Point((-79.34144, 22.12434))
polygon = shape(first_radius['geometry'])

In [166]:
point_coords = (point.coords.xy[0][0], point.coords.xy[1][0])

In [167]:
polygon_dist = [geodesic(point_coords, i).miles for i in polygon.exterior.coords]
min(polygon_dist)

4.193028568573637

In [216]:
# ideally get center data
dist = geodesic(point_coords, tuple(path_features[3]['geometry']['coordinates'])).miles
dist

107.13764477251435

In [235]:
# build simple model
def calc_chance_destroy(point, polygon, path_features):
    for i in range(1, len(path_features)):
        print('Checking day 1')
        print(path_features[i]['geometry']['coordinates'])

calc_chance_destroy(point, polygon, path_features)
# (1 - np.mean([dist, 350])/350) * 0.5 + 0.5 * polygon.contains(point) + 

[-73.8, 21.7]
[-75.7, 22.1]
[-77.8, 22.6]
[-79.4, 23.3]
[-80.4, 24.5]
[-81.5, 28.0]
